In [1]:
import pandas as pd
import numpy as np
import os
import celloracle as co
from celloracle import motif_analysis as ma
from genomepy import Genome, install_genome, config
from gimmemotifs.motif import default_motifs
import mudata as mu
import argparse

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


/cellar/users/aklie/opt/miniconda3/envs/test_celloracle/lib/python3.10/site-packages/loompy/bus_file.py:68: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def twobit_to_dna(twobit: int, size: int) -> str:
/cellar/users/aklie/opt/miniconda3/envs/test_celloracle/lib/python3.10/site-packages/loompy/bus_file.py:85: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def dn

In [ ]:
gdir = "/cellar/users/aklie/opt/gene_program_evaluation/src/inference/grn_models/celloracle/resources/genomes"

In [2]:
path_data = "/cellar/users/aklie/data/datasets/neurips2021_small/annotation/2023_12_05/mdata.h5mu"
path_p2g = "/cellar/users/aklie/data/datasets/neurips2021_small/analysis/celloracle/2024_05_01/processed_peak_file.csv"
organism = "human"
fpr = 0.02
blen = 200
tfb_thr = 10
path_out =  "/cellar/users/aklie/data/datasets/neurips2021_small/analysis/celloracle/2024_05_01/celloracle.tfb.csv"

In [3]:
# Determine genome
if organism == 'human':
    genome = 'hg38'
elif organism == 'mouse':
    genome = 'mm10'

In [8]:
#install_genome(name=genome, genomes_dir=gdir, provider="UCSC")

In [9]:
# Load annotated peak data.
peaks = pd.read_csv(path_p2g)
if peaks.shape[0] == 0:
    tfb = pd.DataFrame(columns=['cre', 'tf', 'score'])
    tfb.to_csv(path_out, index=False)
    exit()
peaks['cre'] = peaks['cre'].str.replace('-', '_')

In [10]:
def decompose_chrstr(peak_str):
    """
    Args:
        peak_str (str): peak_str. e.g. 'chr1_3094484_3095479'

    Returns:
        tuple: chromosome name, start position, end position
    """

    *chr_, start, end = peak_str.split("_")
    chr_ = "_".join(chr_)
    return chr_, start, end

def check_peak_format(peaks_df, gname, gdir):
    """
    Check peak format.
     (1) Check chromosome name.
     (2) Check peak size (length) and remove sort DNA sequences (<5bp)

    """

    df = peaks_df.copy()
    df = df.rename(columns={'cre': 'peak_id', 'gene': 'gene_short_name'})
    n_peaks_before = df.shape[0]

    # Decompose peaks and make df
    decomposed = [decompose_chrstr(peak_str) for peak_str in df["peak_id"]]
    df_decomposed = pd.DataFrame(np.array(decomposed), index=peaks_df.index)
    df_decomposed.columns = ["chr", "start", "end"]
    df_decomposed["start"] = df_decomposed["start"].astype(int)
    df_decomposed["end"] = df_decomposed["end"].astype(int)

    # Load genome data
    genome_data = Genome(gname, genomes_dir=gdir)
    all_chr_list = list(genome_data.keys())

    # DNA length check
    lengths = np.abs(df_decomposed["end"] - df_decomposed["start"])

    # Filter peaks with invalid chromosome name
    n_threshold = 5
    df = df[(lengths >= n_threshold) & df_decomposed.chr.isin(all_chr_list)]

    # DNA length check
    lengths = np.abs(df_decomposed["end"] - df_decomposed["start"])

    # Data counting
    n_invalid_length = len(lengths[lengths < n_threshold])
    n_peaks_invalid_chr = n_peaks_before - df_decomposed.chr.isin(all_chr_list).sum()
    n_peaks_after = df.shape[0]

    # Print
    print("Peaks before filtering: ", n_peaks_before)
    print("Peaks with invalid chr_name: ", n_peaks_invalid_chr)
    print("Peaks with invalid length: ", n_invalid_length)
    print("Peaks after filtering: ", n_peaks_after)

    return df

In [11]:
peaks = check_peak_format(peaks, gname=genome, gdir=gdir)
peaks

Peaks before filtering:  126
Peaks with invalid chr_name:  0
Peaks with invalid length:  0
Peaks after filtering:  126


,peak_id,gene_short_name,score
0,chr1_107964928_107965436,VAV3,1.000000
1,chr1_167630023_167630677,RCSD1,1.000000
2,chr1_224433432_224434024,WDR26,1.000000
3,chr1_231626280_231627069,DISC1,1.000000
4,chr1_44785971_44786863,ZSWIM5,0.629032
...,...,...,...
121,chr8_66712260_66713148,SGK3,1.000000
122,chr8_66774787_66775705,SGK3,1.000000
123,chrX_134796760_134797638,FAM122B,1.000000
124,chrX_134806776_134807626,FAM122B,0.460431


In [12]:
# Instantiate TFinfo object
tfi = ma.TFinfo(
    peak_data_frame=peaks,
    ref_genome=genome,
    genomes_dir=gdir,
)

In [13]:
# Filter motifs based on gene expression
motifs = default_motifs()
dic_motif2TFs = ma.tfinfo_core._get_dic_motif2TFs(
    species='Human',
    motifs=motifs,
    TF_evidence_level='direct_and_indirect',
    formatting='auto'
)
genes = mu.read(os.path.join(path_data, 'rna')).var.index.values.astype('U')
m_keys = []
for m in dic_motif2TFs:
    tfs = np.unique(dic_motif2TFs[m]).astype('U')
    if np.any(np.isin(tfs, genes)):
        m_keys.append(m)
motifs = [m for m in motifs if m.id in m_keys]

In [16]:
# Update config
config.config.config['genomes_dir'] = gdir

In [17]:
# Scan
tfi.scan(
    background_length=blen,
    fpr=fpr,
    motifs=motifs,  # Use filtered motifs
    verbose=True,
    n_cpus=os.cpu_count(),
)

Checking your motifs... Motifs format looks good. 

Initiating scanner... 

Calculating FPR-based threshold. This step may take substantial time when you load a new ref-genome. It will be done quicker on the second time. 

Motif scan started .. It may take long time.



scanning:   0%|          | 0/125 [00:00<?, ? sequences/s]

In [18]:
# Do filtering
tfi.filter_motifs_by_score(threshold=tfb_thr)

Filtering finished: 4410 -> 906


In [19]:
# Extract filtered TF predictions
df = tfi.scanned_filtered[["seqname", "motif_id", "score"]].copy()
df['motif_values'] = df['motif_id'].map(tfi.dic_motif2TFs)
df = df.explode('motif_values')
df = df[df['motif_values'].isin(genes)]  # Filter TFs not in GEX
df = df.groupby(['seqname', 'motif_values'])['score'].max().reset_index()
df = df[['seqname', 'motif_values', 'score']].dropna()
df = df.reset_index(drop=True).rename(columns={'seqname': 'cre', 'motif_values': 'tf'})
df['cre'] = df['cre'].str.replace('_', '-')
df = df.sort_values(['cre', 'score'], ascending=[True, False])

In [21]:
df.to_csv(path_out, index=False)

# Write command to run

```bash
python /cellar/users/aklie/opt/gene_program_evaluation/src/inference/grn_models/celloracle/workflow/scripts/TF2r.py \
-d /cellar/users/aklie/data/datasets/neurips2021_small/annotation/2023_12_05/mdata.h5mu \
-p /cellar/users/aklie/data/datasets/neurips2021_small/analysis/celloracle/2024_05_01/processed_peak_file.csv \
-o human \
-f 0.02 \
-b 200 \
-t 10 \
-q /cellar/users/aklie/data/datasets/neurips2021_small/analysis/celloracle/2024_05_01/celloracle.tfb.csv
```

# DONE!

---